In [1]:
import numpy as np
import json
import tensorflow as tf
import string
import os

/Users/I760138/Vansh/personal/chess-bot/java-chess-engine/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
piece_types = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}

scale = 400

In [ ]:
def encode_fen(fen):
    planes = np.zeros((12, 8, 8), dtype=np.float32)
    parts = fen.split()
    board_rows = parts[0].split('/')
    for r, row in enumerate(board_rows):
        f = 0
        for char in row:
            if char.isdigit():
                f += int(char)
            elif char in piece_types:
                idx = piece_types[char]
                planes[idx, r, f] = 1
                f += 1
    flat = planes.flatten()

    # Side to move
    stm = 1.0 if parts[1] == 'w' else 0.0

    # Castling rights
    castling = [1.0 if x in parts[2] else 0.0 for x in 'KQkq']

    # En passant
    ep = [0.0] * 8
    if parts[3] != '-':
        file = ord(parts[3][0]) - ord('a')
        ep[file] = 1.0

    return np.concatenate([flat, [stm], castling, ep])

def sigmoid(x):
    return 1 / (1 + np.exp(-x / scale))

def inverse_sigmoid(y):
    y = np.clip(y, 1e-7, 1 - 1e-7)  # Avoid division by zero
    return -scale * np.log((1 / y) - 1)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(781,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')  # Regression output
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

if os.path.exists("chess_nn_model.json"):
    with open("chess_nn_model.json", "r") as f:
        weights_list = json.load(f)
    weights = [np.array(w) for w in weights_list]
    model.set_weights(weights)
    print("Loaded model weights from chess_nn_model.json")
else:
    for i in string.ascii_lowercase:
        flag = True
        for j in string.ascii_lowercase:
            filename = f"./db/train_{i}{j}"
            if not os.path.exists(filename):
                flag = False
                break

            print(f"Processing {filename}...")

            X, y = [], []
            with open(filename) as f:
                for line in f:
                    obj = json.loads(line)
                    fen = obj['fen']
                    cp = obj['evals'][0]['pvs'][0].get('cp')
                    if cp is None:  # skip mates for now
                        continue
                    X.append(encode_fen(fen))
                    y.append(cp)

            print(f"Loaded {len(X)} samples")

            X = np.array(X)
            y = np.array(y)
            print("Starting training...")
            model.fit(X, y, batch_size=1024, epochs=10, validation_split=0.2, verbose=1)

        if not flag:
            break


Processing ./db/train_aa...
Loaded 828968 samples
Starting training...
Epoch 1/10
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 2205237.5000 - mae: 329.7458 - val_loss: 1818333.6250 - val_mae: 371.1371
Epoch 2/10
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1676420.5000 - mae: 339.9630 - val_loss: 1719112.0000 - val_mae: 343.9999
Epoch 3/10
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1582131.8750 - mae: 320.1330 - val_loss: 1701763.2500 - val_mae: 338.5207
Epoch 4/10
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1556969.2500 - mae: 313.8789 - val_loss: 1695735.6250 - val_mae: 335.4497
Epoch 5/10
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1549267.1250 - mae: 312.1643 - val_loss: 1693860.5000 - val_mae: 335.4548
Epoch 6/10
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1496831.3750 - mae: 307.2703 - val_loss: 1690985.5000 - val_mae: 333.1010
Epoch 7/10
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1520779.0000 - mae: 308.7988 - val_loss: 1687164.8750 - val_mae: 

In [ ]:
model.summary()
example_fen = "rb3rk1/1p3ppp/pn2p3/1Q6/8/P1NP4/1Pq2PPP/R1B2RK1 w - - 0 17"
example_input = encode_fen(example_fen).reshape(1, -1)
pred = model.predict(example_input)
print(f"Predicted evaluation for example position: {pred[0][0]:.2f}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │       200,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650,117 (2.48 MB)

 Trainable params: 216,705 (846.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 433,412 (1.65 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted evaluation for example position: -117.77


In [6]:
# save model weights to file
weights = model.get_weights()
weights_list = [w.tolist() for w in weights]
with open("chess_nn_model.json", "w") as f:
    json.dump(weights_list, f)